Prompt Construction:
Choose a small set of emotionally neutral or ambiguous phrases.
Translate each phrase directly into both Italian and English.
Use these as prompts for the LLM.
LLM Response Collection:
Query the LLM with each prompt.
Collect its full response in both languages.

Emotion Scoring:
Use emos_eng.zscores() and emos_it.zscores() (from EmoAtlas) to compute the emotional intensity vector for each response.

Expand these vectors into separate columns (e.g., anger, joy, trust, etc.) to compare emotion-by-emotion.
Comparison & Visualization:
Compare scores across languages.
Optionally visualize via bar plots or radar charts for easy interpretation.


In [1]:
!pip install git+https://github.com/MassimoStel/emoatlas
!python -m spacy download en_core_web_lg
!python -m spacy download it_core_news_lg

import requests  # For sending HTTP requests
import json      # For handling JSON data
import matplotlib.pyplot as plt
from emoatlas import EmoScores
import emoatlas
import time
import re
import nltk

  Cloning https://github.com/MassimoStel/emoatlas to /private/var/folders/73/tnmscj2j2k1d5vykfvr7gp7r0000gn/T/pip-req-build-06go90mv
  Running command git clone --filter=blob:none --quiet https://github.com/MassimoStel/emoatlas /private/var/folders/73/tnmscj2j2k1d5vykfvr7gp7r0000gn/T/pip-req-build-06go90mv
  Resolved https://github.com/MassimoStel/emoatlas to commit 7b8133769fed0f948740d6f02f9bed7ed0528424
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.1 MB/s eta 0:00:0000:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.9/567.9 MB 4.9 MB/s eta 0:00:0000:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_lg')


In [2]:
API_KEY = "671n3ZdhWoz3H28F7XVz7UZjLvieH5jI"
API_URL = "https://api.mistral.ai/v1/chat/completions"

#available models ('mistral-tiny', 'mistral-small', 'mistral-medium')
MODEL = "mistral-small"

# Headers for authentication and content type
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

def chat_with_mistral(messages):

    payload = {
        "model": MODEL,
        "messages": messages,
        "temperature": 0.7,     # Creativity level (0 = deterministic, 1 = more random)
        "top_p": 1.0,           # Nucleus sampling parameter
        "stream": False         # Disable streaming for simple usage
    }

    response = requests.post(API_URL, headers=HEADERS, data=json.dumps(payload))
    if response.status_code != 200:
        raise Exception(f"Request failed: {response.status_code} - {response.text}")

    response_data = response.json()

    return response_data['choices'][0]['message']['content']

In [3]:
chat_history_eng = [
    {"role": "user",
     "content":"I want to make multiple prompts to an LLM with the same phrase in english and italian, and I want to use a json file. I need 70 phrases, each needs to ask for opinions. Can you make that json for me, with all the 70 phrases?"}
]

try:
    response = chat_with_mistral(chat_history_eng)
    print(response)
except Exception as e:
    print("Error:", str(e))

Sure, here is a JSON file with 70 prompts for getting opinions in English and Italian:

```json
{
  "prompts": [
    {
      "english": "What is your opinion on vegetarianism?",
      "italian": "Qual è la tua opinione sul vegetarianismo?"
    },
    {
      "english": "Do you think climate change is a serious issue?",
      "italian": "Pensi che il cambiamento climatico sia un problema serio?"
    },
    {
      "english": "What is your opinion on the death penalty?",
      "italian": "Qual è la tua opinione sulla pena di morte?"
    },
    {
      "english": "How do you feel about remote work?",
      "italian": "Come ti senti riguardo al lavoro remoto?"
    },
    {
      "english": "What is your opinion on artificial intelligence?",
      "italian": "Qual è la tua opinione sull'intelligenza artificiale?"
    },
    {
      "english": "Do you think social media has a positive or negative impact on society?",
      "italian": "Pensi che i social media abbiano un impatto positivo o ne

In [ ]:
def clean_response_json(response):
  match = re.search(r'```json\s*(.*?)\s*```', response, re.DOTALL)
  if match:
    json_content = match.group(1)
    return json_content
  else:
    print("No JSON content found in the text.")

json_res = json.loads(clean_response_json(response))
json_res

{'phrases': [{'english': 'What do you think about the new restaurant in town?',
   'italian': 'Cosa ne pensi del nuovo ristorante in città?'},
  {'english': 'Do you like the new movie that just came out?',
   'italian': 'Ti piace il nuovo film che è appena uscito?'},
  {'english': "What's your opinion on the current political situation?",
   'italian': 'Qual è la tua opinione sulla situazione politica attuale?'},
  {'english': 'How do you feel about remote work?',
   'italian': 'Come ti senti riguardo al lavoro remoto?'},
  {'english': "What's your take on the use of artificial intelligence?",
   'italian': "Qual è la tua opinione sull'uso dell'intelligenza artificiale?"},
  {'english': 'Do you have any thoughts on the education system?',
   'italian': 'Hai qualche pensiero sul sistema educativo?'},
  {'english': "What's your opinion on the legalization of marijuana?",
   'italian': 'Qual è la tua opinione sulla legalizzazione della marijuana?'},
  {'english': 'How do you feel about th

Iterate the prompts and get the responses on a new json.

In [9]:
print(len(json_res["phrases"]))

for item in json_res["phrases"]:
    time.sleep(1)
    item["english_response"] = chat_with_mistral([{"role": "user", "content": item["english"]}])

    time.sleep(1)
    item["italian_response"] = chat_with_mistral([{"role": "user", "content": item["italian"]}])


49


In [9]:
output_file = "responses.txt"

with open(output_file, "w") as f:
    f.write("Responses:\n\n")

# Iterate through the prompts and write the responses to the file
for item in json_res["prompts"]:
    english_response = item.get("english_response", "No response generated")
    italian_response = item.get("italian_response", "No response generated")

    with open(output_file, "a") as f:
        f.write(f"English Response:\n{english_response}\n\n")
        f.write(f"Italian Response:\n{italian_response}\n\n")

Apparently spacy has no czech, we can ask for a solution maybe.

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
emos_eng = EmoScores(language="english")
emos_it = EmoScores(language="italian")


In [ ]:
emotions = ["anger", "joy", "trust", "sadness", "disgust", "fear", "anticipation", "surprise"]


# Function to extract zscores for each emotion
def extract_emotions(df, col_name, model, prefix):
    return pd.DataFrame({
        f"{prefix}_{emotion}": df[col_name].apply(lambda x: model.zscores(x).get(emotion, 0))
        for emotion in emotions
    })
df = pd.DataFrame(json_res["prompts"])

# Extract emotion scores
df_eng = extract_emotions(df, "english_response", emos_eng, "eng")
df_it = extract_emotions(df, "italian_response", emos_it, "it")

# Combine into one DataFrame
df_emotions = pd.concat([df_eng, df_it], axis=1)

In [ ]:
df_emotions.to_csv("/df_emotions_full_ds.csv")